In [1]:
import pandas as pd 
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdChemReactions
from rdkit.Chem import AllChem
import os
import PySimpleGUI as sg
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns


matplotlib.use('TkAgg')

polymer_funct_df = pd.read_csv("CSV/polymer_functional_groups.csv",encoding ="shift-jis")#ポリマーデータの読み込み
polymer_funct_df=polymer_funct_df.drop(["Y value","logP","MR"],axis = 1) #不要列の削除
additives_df = pd.read_csv("CSV/additives.csv",encoding ="shift-jis")

polymer_names = polymer_funct_df["Name"].tolist() 
additive_names = additives_df["Name"].tolist()

In [3]:


# GUIのレイアウト
sg.theme("DarkBlack1")

layout = [[sg.Text('Brend', font=('Constantia',20,"bold"))],
          [sg.Text("")],
          [sg.Button("実行",size = (11,1))],
          [sg.Text('_'  * 70)], #横線区切り
          [sg.Text("サンプル名",size=(15, 1)), 
           sg.Input(size=(50, 1),
                    key = "sample_name",text_color='black',background_color='honeydew')],
          [sg.Text("Parameter",size=(15, 1)), 
           sg.Input(size=(50, 1),key = "val",text_color='black',background_color='honeydew')],
          [sg.Text('_'  * 70)], #横線区切り
          [sg.Text('主剤',size=(20,1)),sg.Text('仕込み量',size=(20,1))],
          [sg.Column(layout=[
                  [sg.Combo(polymer_names, size=(20, 1),key=f'main_{n}',text_color='black',background_color='honeydew'),
                   sg.Input("0", size=(20, 1), key=f"main_feed_{n}",text_color='black',background_color='honeydew')
                  ] for n in range(3)],
                     size=(400, 100)  # 列全体のサイズ
                    )
          ],
          [sg.Text('_'  * 70)], #横線区切り
          [sg.Text('添加剤',size=(20,1)),sg.Text('仕込み量',size=(20,1))],
                    [sg.Column(layout=[
                  [sg.Combo(additive_names, size=(20, 1),key=f'ad_{n}',text_color='black',background_color='honeydew'),
                   sg.Input("0", size=(20, 1), key=f"ad_feed_{n}",text_color='black',background_color='honeydew')
                  ] for n in range(5)],
                     size=(400,200)  # 列全体のサイズ
                    )
          ],
          ]


           
# ウインドウの出現位置を指定
win_location = (0, 0)

# モニターの解像度を取得
screen_width, screen_height = sg.Window.get_screen_size()

# ウィンドウのサイズをモニターの半分に設定
window_size = (screen_width,int(screen_height*0.9))

window = sg.Window("Polymers", 
                   layout, 
                   size=window_size,
                   resizable=True,
                   location=win_location)


while True:
    
    event, values = window.read()
    
    if event == sg.WINDOW_CLOSED:
        break
        
    if event == "実行":

        #出力用dict
        composition ={}

        #組成記録用dictにSample名を記録
        composition["Name"]= values["sample_name"]
        composition["Y value"]= values["val"]
        

        for i in range(3):
            main_name = values[f"main_{i}"]
            
            #materialnameに入力されている場合のみデータ参照可能
            if main_name != "":  
                #compositionに組成情報を記録
                composition[main_name]=float(values[f"main_feed_{i}"])
                
        #目的変数値の記録
        composition["Name"] = str(values["sample_name"])
        #目的変数値の記録
        composition["Y value"] = values["val"]
        new_name = values["sample_name"]



    if event == "データ読込":

        edit_sample = values["sample_name"] #検索値をedit_sampleとして格納
        target_row = composition_df.loc[composition_df["Name"] == edit_sample] #検索値と一致する行を取得
        target_row_tr = target_row.transpose() #target_rowのdfを転置
        target_row_dict = target_row_tr.to_dict() #dictのdictとして得られる。keyはindex
        index_number = list(target_row_dict.keys())[0] #indexを数値として取得
        target_row_dict = target_row_dict[index_number] #dictの必要な部分のみ取り出して再代入
        
        window["val"].update(target_row_dict["Y value"])

        #GUIのインプットテーブルを初期化
        for i in range(13):
            window[f"material_{i}"].update("")
            window[f"feed_{i}"].update(0)
        
        #GUIのインプットテーブルの値の更新
        i = 0
        for key in list(target_row_dict.keys())[2:]:
            #値が0ではない時のみ表示
            if target_row_dict[key] !=0:
                window[f"material_{i}"].update(key)
                window[f"feed_{i}"].update(target_row_dict[key])
                i = i + 1 #次の行

    
    if event == "削除":
        try:
            delete_sample = values["sample_name"] #検索値をdelete_sampleとして格納
            composition_df = composition_df[composition_df["Name"] != delete_sample] #検索値と一致する行を削除
            functional_df = functional_df[functional_df["Name"] != delete_sample] 

            
            #削除によって空になったカラムは削除
            composition_df = composition_df.loc[:, (composition_df != 0).any(axis=0)]
            #記録リストに記録
            window["output_table"].update(composition_df.to_numpy().tolist())

            functional_df.to_csv("CSV/polymer_functional_groups.csv",index=False,encoding ="shift-jis")
            composition_df.to_csv("CSV/polymer_composition.csv",index=False,encoding ="shift-jis")
        
        except:
            pass

    if event == "グラフ描画":
            
        num_rows = 4
        num_cols = 5
        
        # ループで10個のサブプロットを生成して表示
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))
        
        for ax, elem in zip(axes.flat,functional_df.columns[2:]):
            x = functional_df[elem].tolist()
            y = functional_df["Y value"].tolist()
            ax.scatter(x, y, s=3, marker='o', color='blue')  # 仮のデータをプロット
            ax.set_title(elem,fontsize=10)
            ax.set_ylabel('Y value',fontsize=10)
        # サブプロット間のスペースを調整
        plt.tight_layout()
        plt.get_current_fig_manager().window.wm_geometry("+0+0")
        plt.show(block=False)


window.close()



In [4]:
composition

{'Name': '', 'Y value': '', 'sample1': 10.0, 'sample3': 90.0}